In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import SPSA
from qiskit_ibm_runtime import EstimatorV2
# from qiskit.primitives import Estimator
from scipy.optimize import minimize
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import Options
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [2]:
#setting up Qiskit_Runtime for Session management
QiskitRuntimeService.save_account(channel="ibm_quantum",token="5bdd6c2a73e84f421422bf22890663ce0c83b96215f059ce6d1af3d00218c1611418dc4cf86a33554518d39a5749435c45ecbe558d19ac2c3013ec978207840c",overwrite=True)
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=True)
backend.name

'ibmq_qasm_simulator'

In [3]:
#formed from the system, the operator composed of the tensor produuct of the pauli matrices
Hamiltonian = SparsePauliOp.from_list([("XZ", -2.0), ("ZX", -1.0), ("YY", 1.0), ("II", 1.0)])

#analyitcally computed the minimum eigenvalue of the Hamiltonian (not possible for large systems)
min_eigval = min(np.linalg.eigvalsh(Hamiltonian.to_matrix()))

#Estimator object to estimate the cost function (expectation value of the Hamiltonian)
estimator = EstimatorV2(backend=backend)
# estimator = Estimator()

#ansatz to apply to the operator for form the expectation value: <psi|H|psi> (this is psi)
ansatz = EfficientSU2(Hamiltonian.num_qubits, reps=1)

#randomly chosen initial paramters for the ansatz
x0  = 2 * np.pi * np.random.rand(ansatz.num_parameters)

#callback function to save and later plot the results
callback = lambda nfev, params, fval, step, acc: print(f"Evaluation: {nfev}: {fval}")

#classical optimizer which takes the cost results and changes the parameters of the ansatz
optimizer = SPSA(maxiter=250, callback=callback)

In [4]:
#Converting both H and the ansatz to the backend's layout as I was runnning into some issues
pass_manager = generate_preset_pass_manager(target=backend.target, optimization_level=3)
ISA_ansatz = pass_manager.run(ansatz)
ISA_hamiltonian = Hamiltonian.apply_layout(layout=ISA_ansatz.layout)

In [9]:
options_dict = {
    "default_shots": 1024,
    "optimization_level": 1,
}



# def cost_func(params, hamiltonian, ansatz, estimator):

#     # pub_access = (ansatz,[hamiltonian],[params])
#     pub_access = [(ISA_ansatz, ISA_hamiltonian, x0)]
#     result = estimator.run(pubs=[pub_access]).result()
#     energy = result[0].data.evs[0]
#     return energy

In [13]:
#IBMQ Session
#multiple shots which begin after the first run is processed
#so you dont have to wait 100 hours, etc
#pub is the variables fed to the estimator
with Session(service=service,backend=backend) as session:
    estimator = EstimatorV2(session=session,options=options_dict)
    pub = [(ISA_ansatz, ISA_hamiltonian, x0)]
    # cost_func = estimator.run(pub)
    # cost_func = lambda params: estimator.run(pub).result().values[0]
    job = estimator.run(pub)
    job_result = job.result()
    pub_result=job_result[0]
    # job_result = cost_func(estimator)
    # pub_result = job_result.
    # result = optimizer.minimize(cost_func, x0=x0)
    # print(session.details())

    



print(result)

/Users/jonahsachs/opt/miniconda3/envs/myQenv/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Cloud simulators will be deprecated on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


TypeError: unsupported operand type(s) for -: 'RuntimeJobV2' and 'RuntimeJobV2'

In [14]:
pub_result = cost_func.result()[0]
print(f"Expectation values: {pub_result.data.evs}")

AttributeError: 'function' object has no attribute 'result'

In [ ]:
print(result)

NameError: name 'result' is not defined

In [ ]:
# import numpy as np
# from qiskit.quantum_info import SparsePauliOp
# from qiskit_algorithms.optimizers import SPSA
# from qiskit.circuit.library import EfficientSU2
# from qiskit.primitives import Estimator
# from scipy.optimize import minimize
# from qiskit_ibm_runtime import QiskitRuntimeService, Session
# from qiskit_ibm_runtime import Options
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Hamiltonian = SparsePauliOp.from_list([("XZ", -2.0), ("ZX", -1.0), ("YY", 1.0), ("II", 1.0)])

# min_eigval = min(np.linalg.eigvalsh(Hamiltonian.to_matrix()))

# estimator = Estimator()

# ansatz = EfficientSU2(Hamiltonian.num_qubits, reps=1)

# x0  = 2 * np.pi * np.random.rand(ansatz.num_parameters)

# callback = lambda nfev, params, fval, step, acc: print(f"Evaluation: {nfev}: {fval}")
# optimizer = SPSA(maxiter=250, callback=callback)

# QiskitRuntimeService.save_account(channel="ibm_quantum",token="5bdd6c2a73e84f421422bf22890663ce0c83b96215f059ce6d1af3d00218c1611418dc4cf86a33554518d39a5749435c45ecbe558d19ac2c3013ec978207840c",overwrite=True)
# service = QiskitRuntimeService()
# backend = service.least_busy(operational=True, simulator=False)

# options = Options(environment={"log_level": "DEBUG"}, execution={"shots": 1024, "init_qubits": True})
# options.execution.init_qubits = True #initialize qubits to |0>
# options.execution.shots = 1024 #number of shots
# # options.environment.callback = [] #later error cancellation
# # options.environment.job_tags = []
# options.environment.log_level = 'DEBUG'


# options_dict = {
#     "execution": {
#         "init_qubits": options.execution.init_qubits,
#         "shots": options.execution.shots,
#     },
#     "environment": {
#         # "callback": options.environment.callback,
#         # "job_tags": options.environment.job_tags,
#         "log_level": options.environment.log_level,
#     }
# }


# #ISA architecture
# pass_manager = generate_preset_pass_manager(target=backend.target, optimization_level=3)
# ISA_ansatz = pass_manager.run(ansatz)
# ISA_hamiltonian = Hamiltonian.apply_layout(layout=ISA_ansatz.layout)



# with Session(backend=backend) as session:
#     estimator = Estimator(session=session)
#     cost_func = lambda params: estimator.run(ansatz, Hamiltonian, parameter_values=params).result().values[0]
#     result = optimizer.minimize(cost_func, x0=x0)

# print(result)

In [ ]:
session.close()